In [ ]:
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

print("IMPORT")
import tensorflow as tf #tensorflow
print(tf.__version__)
import numpy as np #numpy > save loss .. 
from collections import OrderedDict #layer ..
import os, random #dir, random..
import pickle #save & load

# cifar 10 
#wget http://www.cs.toronto.edu/~kriz/cifar-10-python.tar.gz
#tar xzf cifar-10-python.tar.gz

#resnet
#https://arxiv.org/abs/1512.03385

In [ ]:
sparse_softmax_cross_entropy_with_logitsprint("VARIABLES")
#path
DATA_PATH = 'cifar10/cifar-10-batches-py/'
CHECKPOINT_PATH = 'checkpoint/simple_res_net/simple_res_net.ckpt'
SAVE_PATH = 'checkpoint/simple_res_net/'

#parameters
INPUT_SIDE = 32
INPUT_SIZE = INPUT_SIDE * INPUT_SIDE
N_CHANNEL = 3
BATCH_SIZE = 10
EPOCHS = 1000
LR = 0.01
LR_DECAY_RATE = 0.5

LIST_CLASS=['airplane', 'automobile', 'birds', 'cat', 'deer', 'dog', 'frog', 'horse', 'ship', 'truck']
N_CLASSES = len(LIST_CLASS)

#train&test batch
train_file=['data_batch_1','data_batch_2', 'data_batch_3', 'data_batch_4', 'data_batch_5']
test_file=['cifar10/cifar-10-batches-py/test_batch']

#input & output 
x = tf.placeholder("float", [BATCH_SIZE, INPUT_SIZE * N_CHANNEL]) #batch x image size
y = tf.placeholder("float", [BATCH_SIZE, N_CLASSES]) #batch x class`


print("input image size : {}".format(INPUT_SIZE))
print("image channel : {}".format(N_CHANNEL))
print("batch size : {}".format(BATCH_SIZE))
print("num of class : {}".format(N_CLASSES))
print("training epochs : {}".format(EPOCHS))
print("learning rate : {}".format(LR))
print("learning decay rate : {}".format(LR_DECAY_RATE))

In [ ]:
#NETWORK PARAMETERS

print("NETWORK PARAMETERS")

stddev = 0.1

with tf.device("/cpu:0"):
    weights = {
        'conv' : tf.Variable(tf.random_normal([3, 3, N_CHANNEL, 8], stddev=stddev)),
        'conv1_1x1' : tf.Variable(tf.random_normal([1, 1, 8, 2], stddev=stddev)),
        'conv1_3x3' : tf.Variable(tf.random_normal([3, 3, 2, 2], stddev=stddev)),
        'conv1_1x1_16' : tf.Variable(tf.random_normal([1, 1, 2, 8], stddev=stddev)),
        'conv2_1x1' : tf.Variable(tf.random_normal([1, 1, 8, 2], stddev=stddev)),
        'conv2_3x3' : tf.Variable(tf.random_normal([3, 3, 2, 2], stddev=stddev)),
        'conv2_1x1_16' : tf.Variable(tf.random_normal([1, 1, 2, 8], stddev=stddev)),

        # conv 8 + conv2 8= 16filters 

        'conv3_1x1' : tf.Variable(tf.random_normal([1, 1, 16, 4], stddev=stddev)),
        'conv3_3x3' : tf.Variable(tf.random_normal([3, 3, 4, 4], stddev=stddev)),
        'conv3_1x1_32' : tf.Variable(tf.random_normal([1, 1, 4, 16], stddev=stddev)),
        'conv4_1x1' : tf.Variable(tf.random_normal([1, 1, 16, 4], stddev=stddev)),
        'conv4_3x3' : tf.Variable(tf.random_normal([3, 3, 4, 4], stddev=stddev)),
        'conv4_1x1_32' : tf.Variable(tf.random_normal([1, 1, 4, 16], stddev=stddev)),

        # conv2 지난거 16 + conv4 16 = 32 filters

        'dense1' : tf.Variable(tf.random_normal([16*16*32, 1000], stddev=stddev, name='dense11')),
        'dense2' : tf.Variable(tf.random_normal([1000, N_CLASSES], stddev=stddev, name='dense22'))


    }
    biases = {

        'conv' : tf.Variable(tf.random_normal([8], stddev=stddev)),
        'conv1_1x1' : tf.Variable(tf.random_normal([2], stddev=stddev)),
        'conv1_3x3' : tf.Variable(tf.random_normal([2], stddev=stddev)),
        'conv1_1x1_16' : tf.Variable(tf.random_normal([8], stddev=stddev)),
        'conv2_1x1' : tf.Variable(tf.random_normal([2], stddev=stddev)),
        'conv2_3x3' :tf.Variable(tf.random_normal([2], stddev=stddev)),
        'conv2_1x1_16' : tf.Variable(tf.random_normal([8], stddev=stddev)),

        'conv3_1x1' : tf.Variable(tf.random_normal([4], stddev=stddev)),
        'conv3_3x3' : tf.Variable(tf.random_normal([4], stddev=stddev)),
        'conv3_1x1_32' : tf.Variable(tf.random_normal([16], stddev=stddev)),
        'conv4_1x1' : tf.Variable(tf.random_normal([4], stddev=stddev)),
        'conv4_3x3' : tf.Variable(tf.random_normal([4], stddev=stddev)),
        'conv4_1x1_32' : tf.Variable(tf.random_normal([16], stddev=stddev)),

        'dense1' : tf.Variable(tf.random_normal([1000], stddev=stddev)),
        'dense2' : tf.Variable(tf.random_normal([N_CLASSES], stddev=stddev))
    }

In [ ]:
#model
#http://laonple.blog.me/220764986252 - bottleneck 
#conv 3x3
#conv (1x1, 3x3, 1x1) - relu > 16
#conv (1x1, 3x3, 1x1) relu > 32
#conv (1x1, 3x3, 1x1) relu > 64
#avg pooling 
#fc
#softmax

print("NETWORK FUNCTION")

def SimpleResNet(img_width, img_height, img_channel, _x, _w, _b, scope='SimpleResNet'):
    network = OrderedDict() #network layers

    # X RESHAPE
    _x_r = tf.reshape(_x, shape=[-1,img_width,img_height, img_channel])
    
    with tf.variable_scope(scope):
        conv = tf.nn.conv2d(_x_r, _w['conv'], strides=[1, 1, 1, 1], padding='SAME')
        conv = tf.nn.bias_add(conv, _b['conv'])
        conv = tf.nn.relu(conv)
        network['conv'] = conv
        
        #16
        conv1_1x1 = tf.nn.conv2d(conv, _w['conv1_1x1'], strides=[1, 1, 1, 1], padding='SAME')
        conv1_1x1 = tf.nn.bias_add(conv1_1x1, _b['conv1_1x1'])
        conv1_1x1 = tf.nn.relu(conv1_1x1)
        conv1_3x3 = tf.nn.conv2d(conv1_1x1, _w['conv1_3x3'], strides=[1, 1, 1, 1], padding='SAME')
        conv1_3x3 = tf.nn.bias_add(conv1_3x3, _b['conv1_3x3'])
        conv1_3x3 = tf.nn.relu(conv1_3x3)
        conv1_1x1_16 = tf.nn.conv2d(conv1_3x3, _w['conv1_1x1_16'], strides=[1, 1, 1, 1], padding='SAME')
        conv1_1x1_16 = tf.nn.bias_add(conv1_1x1_16, _b['conv1_1x1_16'])
        conv1_1x1_16 = tf.nn.relu(conv1_1x1_16)
        network['conv1_1x1_16'] = conv1_1x1_16
        
        #16
        conv2_1x1 = tf.nn.conv2d(conv1_1x1_16, _w['conv2_1x1'], strides=[1, 1, 1, 1], padding='SAME')
        conv2_1x1 = tf.nn.bias_add(conv2_1x1, _b['conv2_1x1'])
        conv2_1x1 = tf.nn.relu(conv2_1x1)
        conv2_3x3 = tf.nn.conv2d(conv2_1x1, _w['conv2_3x3'], strides=[1, 1, 1, 1], padding='SAME')
        conv2_3x3 = tf.nn.bias_add(conv2_3x3, _b['conv2_3x3'])
        conv2_3x3 = tf.nn.relu(conv2_3x3)
        conv2_1x1_16 = tf.nn.conv2d(conv2_3x3, _w['conv2_1x1_16'], strides=[1, 1, 1, 1], padding='SAME')
        conv2_1x1_16 = tf.nn.bias_add(conv2_1x1_16, _b['conv2_1x1_16'])
        #print(conv2_1x1_16.shape)
        #32
        conv2_1x1_16 = tf.concat([conv, conv2_1x1_16], 3) 
        conv2_1x1_16 = tf.nn.relu(conv2_1x1_16)
        network['conv2_1x1_16'] = conv2_1x1_16
       
        #print(conv2_1x1_16.shape)
        #32       32 x 32 x 16+16 >>> 
        conv3_1x1 = tf.nn.conv2d(conv2_1x1_16, _w['conv3_1x1'], strides=[1, 1, 1, 1], padding='SAME')
        conv3_1x1 = tf.nn.bias_add(conv3_1x1, _b['conv3_1x1'])
        conv3_1x1 = tf.nn.relu(conv3_1x1)
        conv3_3x3 = tf.nn.conv2d(conv3_1x1, _w['conv3_3x3'], strides=[1, 1, 1, 1], padding='SAME')
        conv3_3x3 = tf.nn.bias_add(conv3_3x3, _b['conv3_3x3'])
        conv3_3x3 = tf.nn.relu(conv3_3x3)
        conv3_1x1_32 = tf.nn.conv2d(conv3_3x3, _w['conv3_1x1_32'], strides=[1, 1, 1, 1], padding='SAME')
        conv3_1x1_32 = tf.nn.bias_add(conv3_1x1_32, _b['conv3_1x1_32'])
        conv3_1x1_32 = tf.nn.relu(conv3_1x1_32)
        network['conv3_1x1_32'] = conv3_1x1_32
        
        conv4_1x1 = tf.nn.conv2d(conv3_1x1_32, _w['conv4_1x1'], strides=[1, 1, 1, 1], padding='SAME')
        conv4_1x1 = tf.nn.bias_add(conv4_1x1, _b['conv4_1x1'])
        conv4_1x1 = tf.nn.relu(conv4_1x1)
        conv4_3x3 = tf.nn.conv2d(conv4_1x1, _w['conv4_3x3'], strides=[1, 1, 1, 1], padding='SAME')
        conv4_3x3 = tf.nn.bias_add(conv4_3x3, _b['conv4_3x3'])
        conv4_3x3 = tf.nn.relu(conv4_3x3)
        conv4_1x1_32 = tf.nn.conv2d(conv4_3x3, _w['conv4_1x1_32'], strides=[1, 1, 1, 1], padding='SAME')
        conv4_1x1_32 = tf.nn.bias_add(conv4_1x1_32, _b['conv4_1x1_32'])
        
        #64
        conv4_1x1_32 = tf.concat([conv2_1x1_16, conv4_1x1_32 ], 3)
        conv4_1x1_32 = tf.nn.relu(conv4_1x1_32)
        network['conv4_1x1_32'] = conv4_1x1_32
        
        pool = tf.nn.avg_pool(conv4_1x1_32, ksize=[1, 2, 2, 1], strides=[1, 2, 2, 1], padding='SAME')
        #pool = tf.nn.avg_pool(conv6_1x1_64, ksize=[1, 2, 2, 1], strides=[1, 2, 2, 1], padding='SAME')
        network['pool'] = pool
        
        dense = tf.reshape(pool, [-1, _w['dense1'].get_shape().as_list()[0]])
        dense1 = tf.add(tf.matmul(dense, _w['dense1']), _b['dense1'])
        dense1 = tf.nn.relu(dense1)
        network['dense1'] = dense1
        
        logit = tf.add(tf.matmul(dense1, _w['dense2']), _b['dense2'])
        network['logit'] = logit
        
    return network

In [ ]:
out = SimpleResNet(INPUT_SIDE, INPUT_SIDE, N_CHANNEL, x, weights, biases, 'SimpleResNet')
print("CHECK LAYERS")
for key, value in out.items():
    print (key, value)

In [ ]:
#save
saver = tf.train.Saver(max_to_keep=3)
save_step = 100 #save for 100 epoch
if not os.path.exists(CHECKPOINT_PATH):
    os.makedirs(CHECKPOINT_PATH)
print ("SAVER READY")

In [ ]:
#cost
cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(labels=y, logits=out['logit']))
tf.summary.scalar("loss", cost)
print("LOSS FUNCTION")

#learning rate
global_step = tf.Variable(0, trainable=False, name="global_step")
learning_rate = tf.train.exponential_decay(learning_rate=LR,
                                           global_step=global_step,
                                           decay_steps=50000,
                                           decay_rate=LR_DECAY_RATE,
                                           staircase=True,
                                           name="learning_rate")
learning_rate = tf.maximum(learning_rate, 0.0001)
tf.summary.scalar("learning_rate", learning_rate)
print("LERANING RATE : {}".format(learning_rate))

#optimizer
adam = tf.train.AdamOptimizer(learning_rate = learning_rate).minimize(cost)
sgd = tf.train.GradientDescentOptimizer(learning_rate = learning_rate).minimize(cost)
print("OPTIMIZER 1 : {}".format(adam))
print("OPTIMIZER 2 : {}".format(sgd))

corr = tf.equal(tf.argmax(out['logit'], 1), tf.argmax(y,1))
accr = tf.reduce_mean(tf.cast(corr, "float"))

In [ ]:
init = tf.global_variables_initializer()
sess = tf.Session()
sess.run(init)


#summary_writer = tf.summary.FileWriter(CHECKPOINT_PATH, sess.graph)
#checkpoint = tf.train.latesrt_checkpoint(CHECKPOINT_PATH)

#포문돌다 세이브할땐
#save_path = saver.save(sess, CHECKPOINT_PATH + "mm.ckpt", global_step=global_step)

In [1]:
import tensorflow as tf
a = tf.Variable(tf.random_normal([8196, 1000], stddev=0.01 ))


SyntaxError: unexpected EOF while parsing (<ipython-input-1-95a999170a05>, line 2)